<a href="https://colab.research.google.com/github/lavstat17/Notebook_Product_scoring/blob/main/iwon_organics_comaprsion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing libraries
import pandas as pd
import math
import warnings
import re,logging
warnings.filterwarnings("ignore")

class product_score:
  def __init__(self):
    self.log = logging.getLogger(__name__)

  def integration(self):
    #Read the Scrapped data Vitaminshoppe and chetoos
    df = pd.read_csv(r'/content/drive/MyDrive/iwon_organics_data.csv', engine='python')
    #convert each serving size to 100g
    df_new = df[['protein_gram','Saturatedfat','sugars_gram','Servingsize_g','sodium_mg','fiber_gram','calories_value']].astype(int)
    d = {14:7.15,28:3.58,42:2.39}
    df_new['constant'] = df['Servingsize_g'].map(d)
    df_new['protein_100g'] = df_new['protein_gram']*df_new['constant'].astype(int)
    df_new['Saturated_Fat_100g'] = df_new['Saturatedfat']*df_new['constant'].astype(int)
    df_new['sugars_100g'] = df_new['sugars_gram']*df_new['constant'].round(decimals=0)
    df_new['fiber_100g'] = df_new['fiber_gram']*df_new['constant'].round(decimals=0)
    df_new['Servingsize_100g'] = df_new['Servingsize_g']*df_new['constant'].round(decimals=0)
    df_new['sodium_mg'] = df_new['sodium_mg'].astype(int)
    df_new['Energy_Kcal'] = df_new['calories_value']*df_new['constant'].round(decimals=0)
    df_new['Energy_KJ'] = df_new['calories_value']*4.184
    df_new.to_csv('conversion_data.csv')

    #negative factors dataframe
    df_new1= df_new[['Energy_KJ','sodium_mg','Saturated_Fat_100g','sugars_100g']]

    ##positive factors dataframe
    df_new2= df_new[['fiber_100g','protein_100g']]

    ## Negative factors scoring based on threshold value
    bins1 = [0,335,670,1005,1340,1675,2010,2345,2680,3015,3350]
    bins2 = [0,1,2,3,4,5,6,7,8,9,10,15]
    bins3 = [0,4.5,9,13.5,18,22.5,27,31,36,40,45]
    bins4 = [0,90,180,270,360,450,540,630,720,810,900,1500]
    df_new1['Energy_KJ_score'] = pd.cut(df_new1['Energy_KJ'], bins=bins1, labels=False,include_lowest=True)
    df_new1['Saturated_Fat_score_'] = pd.cut(df_new1['Saturated_Fat_100g'],bins=bins2, labels=False,include_lowest=True)
    df_new1['sugars_100g_score'] = pd.cut(df_new1['sugars_100g'],bins=bins3,labels=False,include_lowest=True)
    df_new1['sodium_mg_score'] = pd.cut(df_new1['sodium_mg'],bins=bins4,labels=False,include_lowest=True)

    ## Positive factors scoring based on threshold values
    bin1 = [0,0.7,1.4,2.1,2.8,3.5,40]
    bin2 = [0,1.6,3.2,4.8,6.4,8.0,100]
    df_new2['fiber_score'] = pd.cut(df_new2['fiber_100g'], bins=bin1, labels=False,include_lowest=True)
    df_new2['protein_score'] = pd.cut(df_new2['protein_100g'],bins=bin2, labels=False,include_lowest=True)
    return df,df_new1,df_new2


  #calculating A points
  #Total 'A' points = (points for energy) + (points for saturated fat) + (points for sugars) + (points for sodium)
  def A_points(self,row):
    return row['Energy_KJ_score']+row['sodium_mg_score']+row['Saturated_Fat_score_']+row['sugars_100g_score']

  #caluclating C points
  #Total 'C' points = (points for % fruit, vegetable & nut content) + (points for fibre [either NSP or AOAC]) + (points for protein)
  def C_points(self,row):
    return row['fiber_score']+row['protein_score']

  # Based on conditions finding overall score
  def overall_score(self,row):
    if row['A'] < 11:
      return row['A']- row['C']
    elif row['A'] >= 11:
      return row['A'] - row['C']

  def perform_score(self,schedule_app=None):
    df,df_new1,df_new2 = self.integration()
    A = self.A_points(df_new1)
    C = self.C_points(df_new2)
    
    score = {'A':A,'C':C}
    score = pd.concat(score, axis=1)
    score['final_score'] = score.apply( lambda row : self.overall_score(row), axis = 1)
    bin2 = [-15,-2,3,11,16,17]
    labels = ['Green','Yellow','Orange','pink','Red']
    
    #Creating 5CNL format
    score['Tag'] = pd.cut(score['final_score'], bins=bin2, labels=labels)
    score = pd.concat([df[['Nameofproduct','category']],score],axis=1)
    score.to_csv('iwon_comparison_results.csv')
    print(score)


if __name__ == '__main__':
  reco = product_score()
  reco.perform_score()

                                        Nameofproduct  ...     Tag
0                       iwon_organics - Mesquite  BBQ  ...   Green
1                          iwon_organics - Korean BBQ  ...   Green
2                 iwon_organics - spicy sweet peppers  ...   Green
3                        iwon_organics - nacho cheese  ...   Green
4                         iwon_organics - sweet djion  ...   Green
5         iwon_organics - tapatio jalapeno sour cream  ...   Green
6                      iwon_organics - cheddar cheese  ...   Green
7                          iwon_organics - red pepper  ...   Green
8                   iwon_organics - caramelized onion  ...   Green
9               CHEETOS��Puffs Cheese Flavored Snacks  ...  Yellow
10  CHEETOS��Simply Puffs White Cheddar Cheese Fla...  ...  Yellow
11  CHEETOS��Puffs FLAMIN' HOT��Cheese Flavored Sn...  ...  Orange
12            CHEETOS��Crunchy Cheese Flavored Snacks  ...  Yellow
13  CHEETOS��Simply Crunchy White Cheddar Cheese F...  ...  Ye